In [12]:
import random as rd
import itertools

def QuickGamePlays(Strat1,Strat2,LRounds) -> tuple[float,float,int,int]:
    P1State, P2State = [None]*LRounds, [None]*LRounds
    for i in range(LRounds):
        P1Statecopy, P2Statecopy = P1State.copy(), P2State.copy()
        P1State[i] = Strat1(i,P1Statecopy,P2Statecopy)[i]
        P2State[i] = Strat2(i,P2Statecopy,P1Statecopy)[i]
    return P1State, P2State

def RATIO(nthround,P1State,P2State):
    nd = P2State.count("d") # no. of defects
    nc = nthround - nd      # no. of cooperations
    if nd == 0:
        P1State[nthround]="c"
    elif nc/nd > 0.925:
        P1State[nthround]="c"
    else:
        P1State[nthround]="d"
    return P1State

def ScoreTeam(SuperSecretCodes='random'):
    lenCode = 8
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode)))), list(rd.choice(list(itertools.product(['c','d'],repeat=lenCode))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    PaceMakerMoves = SuperSecretCodes[1] + ['c']*(200 - lenCode)
    RunnerMoves = QuickGamePlays(RATIO, (lambda a, b, c : PaceMakerMoves), lenCode)[0] + SuperSecretCodes[0] + ['d']*(200 - 2*lenCode)

    def ScoreRunner(nthround,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:max(lenCode, nthround)] == PaceMakerMoves[:max(lenCode, nthround)]:
            P1State[nthround] = RunnerMoves[nthround]
            return P1State
        
        if 199 - nthround < 1:
            P1State[nthround] = "d"
            return P1State

        # Otherwise, play RATIO
        P1State[nthround] = RATIO(nthround,P1State,P2State)[nthround]
        return P1State

    def ScorePaceMaker(nthround,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:nthround] == RunnerMoves[:nthround]:
            P1State[nthround] = PaceMakerMoves[nthround]
            return P1State
        
        # Otherwise, play ALLD
        P1State[nthround]="d"
        return P1State
    
    return ScoreRunner, ScorePaceMaker

In [10]:
def cTFT(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def dTFT(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def TF2T(round,P1State,P2State):
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" and P2State[round-2]=="d":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def T2FT(round,P1State,P2State):
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" or P2State[round-2]=="d":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def GRIM(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="d" or P2State[round-1]=="d":
        P1State[round]="d"
    else:
        P1State[round]="c"
    return P1State
def GRADUAL(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="c"
    else:
        nDef1=0
        for j in range(nDef2):
            if P1State[round-j-1]=="d":
                nDef1=nDef1+1  
        if nDef2>nDef1:
            P1State[round]="d"
        else:
            P1State[round]="c"
    return P1State
def softMAJ(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="c"
    elif 2*nDef2<=round:
        P1State[round]="c"
    else:
        P1State[round]="d"        
    return P1State
def hardMAJ(round,P1State,P2State):
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    if round==0:
        P1State[round]="d"
    elif 2*nDef2<round:
        P1State[round]="c"
    else:
        P1State[round]="d"       
    return P1State
def PROBER(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    elif round==1:
        P1State[round]="c"
    elif round==2:
        P1State[round]="c" 
    elif P2State[1]=="c" and P2State[2]=="c":
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
    return P1State
def GTFT(round,P1State,P2State):
    cost = 1
    benefit = 2
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
    return P1State
def dWSLS(round,P1State,P2State):
    if round==0:
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
    return P1State
def cWSLS(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
    return P1State
def ALLD(round,P1State,P2State):
    P1State[round]="d"
    return P1State
def ALLC(round,P1State,P2State):
    P1State[round]="c"
    return P1State

In [60]:
from typing import List
import itertools

# generate all n length move combinations
def generateMoves(n):
    moves = list(itertools.product(['c','d'],repeat=n))
    return moves

def generateStrat(moves: List[str]):
    def strat(round, P1State, P2State):
        P1State[round] = moves[round]
        return P1State
    return strat

lencode = 5
moves = [list(move) for move in generateMoves(lencode)]

# play games with default strategies
default_strats = [ALLC, ALLD, cWSLS, dWSLS, GTFT, PROBER, hardMAJ, softMAJ, GRADUAL, GRIM, T2FT, TF2T, dTFT, cTFT]
default_games = [QuickGamePlays(RATIO, strat, lencode) for strat in default_strats]

# remove P2State from moves list
P2States = [game[1] for game in default_games]
filtered_moves = [move for move in moves if move not in P2States]

# play all filtered moves against RATIO
filtered_games = [QuickGamePlays(RATIO, generateStrat(move), lencode) for move in filtered_moves]

score_dict = {
    ("d", "c") : 2,
    ("d", "d") : 0,
    ("c", "c") : 1,
    ("c", "d") : -1
}

score_list = [sum(score_dict[i] for i in zip(game[0], game[1])) for game in filtered_games]

# sort filtered_moves by score_list
sorted_moves = [move for _, move in sorted(zip(score_list, filtered_moves))]

In [61]:
sorted_moves

[['c', 'c', 'd', 'd', 'd'],
 ['c', 'd', 'c', 'd', 'd'],
 ['c', 'd', 'd', 'd', 'd'],
 ['c', 'd', 'd', 'c', 'd'],
 ['d', 'c', 'c', 'd', 'd'],
 ['d', 'c', 'd', 'd', 'd'],
 ['c', 'c', 'c', 'd', 'd'],
 ['c', 'c', 'd', 'c', 'd'],
 ['c', 'c', 'd', 'd', 'c'],
 ['c', 'd', 'c', 'c', 'd'],
 ['c', 'd', 'c', 'd', 'c'],
 ['c', 'd', 'd', 'd', 'c'],
 ['d', 'd', 'c', 'd', 'd'],
 ['d', 'd', 'd', 'c', 'd'],
 ['d', 'd', 'd', 'd', 'c'],
 ['c', 'd', 'd', 'c', 'c'],
 ['d', 'c', 'c', 'c', 'd'],
 ['d', 'c', 'c', 'd', 'c'],
 ['d', 'c', 'd', 'd', 'c'],
 ['d', 'd', 'c', 'c', 'd'],
 ['c', 'c', 'c', 'c', 'd'],
 ['c', 'c', 'c', 'd', 'c'],
 ['c', 'c', 'd', 'c', 'c'],
 ['c', 'd', 'c', 'c', 'c'],
 ['d', 'c', 'd', 'c', 'c'],
 ['d', 'd', 'd', 'c', 'c'],
 ['d', 'd', 'c', 'c', 'c']]